# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('data/nhanes_2015_2016.csv')
df.head(3)

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0


## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [19]:
# get all females between 35-50
female_df = df[(df['RIAGENDR'] == 2) &
               ((df['RIDAGEYR'] >= 35) & df['RIDAGEYR'] <= 50)]
female_df

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
5,83737,2.0,2.0,NaN,2,2,72,1,2.0,2.0,...,122.0,58.0,64.4,150.0,28.6,34.4,33.5,31.4,92.9,NaN
7,83742,1.0,NaN,1.0,2,2,32,1,2.0,4.0,...,114.0,70.0,64.5,151.3,28.2,34.1,33.1,31.5,93.3,2.0
12,83752,1.0,NaN,2.0,1,2,30,2,1.0,4.0,...,104.0,50.0,71.2,163.6,26.6,37.3,35.7,31.0,90.7,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5724,93682,NaN,NaN,NaN,2,2,41,5,1.0,5.0,...,122.0,84.0,58.2,166.9,20.9,37.1,35.3,26.9,80.8,2.0
5727,93689,2.0,1.0,NaN,2,2,69,1,1.0,1.0,...,166.0,64.0,64.8,151.9,28.1,32.2,32.6,28.7,101.1,2.0
5730,93695,2.0,2.0,NaN,1,2,76,3,1.0,3.0,...,112.0,46.0,59.1,165.8,21.5,38.2,37.0,29.5,95.0,2.0
5732,93697,1.0,NaN,1.0,1,2,80,3,1.0,4.0,...,146.0,58.0,71.7,152.2,31.0,31.3,37.5,28.8,NaN,2.0


In [20]:
# replace values for DMDMARTL
female_df['DMDMARTL'] = female_df['DMDMARTL'].replace({
    1: 'Married', 2: 'Not Married', 3: 'Not Married',
    4: 'Not Married', 5: 'Not Married', 6: 'Not Married',
    77: np.nan, 99: np.nan, '.': np.nan}).dropna()

female_df['DMDMARTL'].value_counts()

Not Married    1546
Married        1303
Name: DMDMARTL, dtype: int64

In [21]:
# replace values for DMDEDUC2
female_df['DMDEDUC2'] = female_df['DMDEDUC2'].replace({
    1: 'Not Graduated', 2: 'Not Graduated', 3: 'Not Graduated', 4: 'Not Graduated',
    5: 'Graduated', 7: np.nan, 9: np.nan, '.': np.nan
}).dropna()

female_df['DMDEDUC2'].value_counts()

Not Graduated    2132
Graduated         717
Name: DMDEDUC2, dtype: int64

In [22]:
# proportion of women who graduated
female_df.groupby('DMDMARTL')['DMDEDUC2'].value_counts(normalize=True)

DMDMARTL     DMDEDUC2     
Married      Not Graduated    0.682272
             Graduated        0.317728
Not Married  Not Graduated    0.803883
             Graduated        0.196117
Name: DMDEDUC2, dtype: float64

In [34]:
pd.crosstab(female_df['DMDMARTL'], female_df['DMDEDUC2'])

DMDEDUC2,Graduated,Not Graduated
DMDMARTL,,
Married,414,889
Not Married,303,1242


In [29]:
# calculate proportion and size for married women
prop_married = .317728
n_married    = len(female_df[female_df['DMDMARTL'] == 'Married'])
se_married   = np.sqrt(prop_married * (1 - prop_married) / n_married)
se_married

0.012898354557108836

In [30]:
# calculate bounds
moe = 1.96 * se_married
lower_bound, upper_bound = prop_married - moe, prop_married + moe
print(f'Lower and Upper bounds for married women who have graduated: ({lower_bound}, {upper_bound})')

Lower and Upper bounds for married women who have graduated: (0.2924472250680667, 0.3430087749319333)


In [35]:
# automated functions with statsmodels
sm.stats.proportion_confint(414, 414 + 889)

(0.29244800208510313, 0.34300863644137425)

In [31]:
# calculate proportion and size for not married women
prop_not_married = .196117
n_not_married    = len(female_df[female_df['DMDMARTL'] == 'Not Married'])
se_not_married   = np.sqrt(prop_not_married * (1 - prop_not_married) / n_not_married)
se_not_married

0.010098323939390348

In [33]:
# calculate bounds
moe = 1.96 * se_not_married
lower_bound, upper_bound = prop_not_married - moe, prop_not_married + moe
print(f'Lower and Upper bounds for NOT married women who have graduated: ({lower_bound}, {upper_bound})')

Lower and Upper bounds for NOT married women who have graduated: (0.17632428507879494, 0.2159097149212051)


In [36]:
sm.stats.proportion_confint(303, 303 + 1242)

(0.17631776826939938, 0.2159152414393385)

__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case.

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

## Question 2

Construct a 95% confidence interval for the proportion of smokers who are female. Construct a 95% confidence interval for the proportion of smokers who are male. Construct a 95% confidence interval for the **difference** between those two gender proportions.